# Entrenamiento y Evaluación del Modelo CNN para Melanoma

In [ ]:
import os
import sys

sys.path.append(os.path.abspath('../src'))
#sys.path.append(os.path.abspath('/content/TIC_CNN_Modelo_Melanoma/src')) # Descomentar si se usa Colab

from data.data_loader import load_and_clean_data
from data.split_data import create_splits
from data.preprocessing import create_data_generators, create_data_flow_from_dataframe
from config.config import MODEL_CONFIG, TRAINING_CONFIG, SAMPLE_SIZE
from models.transfer_learning import build_resnet50_classifier
from training.train import TwoStageTrainer
from evaluation.plots import plot_two_stage_training

### 1. Entrenamiento del Modelo

In [ ]:
# Cargar datos procesados
df = load_and_clean_data(sample_size=SAMPLE_SIZE)
train_df, val_df, test_df = create_splits(df)

# Crear generadores de datos
td, vtd = create_data_generators()
train_gen = create_data_flow_from_dataframe(td, train_df)
val_gen = create_data_flow_from_dataframe(vtd, val_df, shuffle=False)
test_gen = create_data_flow_from_dataframe(vtd, test_df, shuffle=False)

# Construir y entrenar modelo
model, base = build_resnet50_classifier(**MODEL_CONFIG)
trainer = TwoStageTrainer(model, base, TRAINING_CONFIG)
history_a = trainer.stage_a_head_training(train_gen, val_gen)
history_b = trainer.stage_b_fine_tuning(train_gen, val_gen)


### 2. Evaluación del Modelo

In [ ]:
test_loss, test_acc, test_auc = trainer.model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.4f}, Test AUC: {test_auc:.4f}, Test Loss: {test_loss:.4f}")

### 3. Visualización de Resultados

In [ ]:

plot_two_stage_training(history_a, history_b)